In [ ]:
from datetime import datetime
import logging

logging.basicConfig(filename='train_initialization.log', filemode='w', format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', level=logging.INFO)
logging.info('SCRIPT INICIADO')


In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.backend import clear_session
from keras.optimizers import SGD
from pathlib import Path
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D
from keras import initializers, regularizers

logging.info('BIBLIOTECAS IMPORTADAS')

In [ ]:
# reusable stuff
import constants
import callbacks
import generators

logging.info('CONFIGURAÇÕES IMPORTADAS')

In [ ]:
# No kruft plz
clear_session()
logging.info('SESSÃO REINICIALIZADA COM SUCESSO')

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

logging.info('AJUSTES DE USO DE GPU FINALIZADO COM SUCESSO')

In [ ]:
# Config
height = constants.SIZES['basic']
width = height
weights_file = "weights.best_mobilenet" + str(height) + ".hdf5"

logging.info('PESOS DO MODELO IMPORTADOS COM SUCESSO')

In [ ]:
conv_base = MobileNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(height, width, constants.NUM_CHANNELS)
)
logging.info('MODELO MobileNetV2 IMPORTADO COM SUCESSO')

In [ ]:
# First time run, no unlocking
conv_base.trainable = False
logging.info('AJUSTE DE TREINAMENTO DO MODELO REALIZADO COM SUCESSO')

In [ ]:
# Let's see it
print('Summary')
print(conv_base.summary())

logging.info('SUMARIO DO MODELO')
logging.info(conv_base.summary())

In [ ]:
# Let's construct that top layer replacement
x = conv_base.output
x = AveragePooling2D(pool_size=(7, 7))(x)
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_initializer=initializers.he_normal(seed=None), kernel_regularizer=regularizers.l2(.0005))(x)
x = Dropout(0.5)(x)
# Essential to have another layer for better accuracy
x = Dense(128,activation='relu', kernel_initializer=initializers.he_normal(seed=None))(x)
x = Dropout(0.25)(x)
predictions = Dense(constants.NUM_CLASSES,  kernel_initializer="glorot_uniform", activation='softmax')(x)

logging.info('NOVAS CAMADAS CONFIGURADAS COM SUCESSO')

In [ ]:
print('Stacking New Layers')
model = Model(inputs = conv_base.input, outputs=predictions)

logging.info('NOVAS CAMADAS ADICIONADAS AO MODELO COM SUCESSO')

In [ ]:
# Load checkpoint if one is found
if os.path.exists(weights_file):
        print ("loading ", weights_file)
        model.load_weights(weights_file)

logging.info('VERIFICAÇÃO DE CHECKPOINT')

In [ ]:
# Get all model callbacks
callbacks_list = callbacks.make_callbacks(weights_file)

logging.info('INICIALIZACAO DO CALLBACK REALIZADO')

In [ ]:
print('Compile model')
# originally adam, but research says SGD with scheduler
# opt = Adam(lr=0.001, amsgrad=True)
opt = SGD(momentum=.9)
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

logging.info('MODELO COMPILADO COM SUCESSO')

In [ ]:
# Get training/validation data via generators
train_generator, validation_generator = generators.create_generators(height, width)

logging.info('BASES DE TREINAMENTO E TESTES IMPORTADAS COM SUCESSO')

In [ ]:
print('Start training!')

logging.info('TREINAMENTO DO MODELO INICIADO')

history = model.fit_generator(
    train_generator,
    callbacks=callbacks_list,
    epochs=constants.TOTAL_EPOCHS,
    steps_per_epoch=constants.STEPS_PER_EPOCH,
    shuffle=True,
    workers=4,
    use_multiprocessing=False,
    validation_data=validation_generator,
    validation_steps=constants.VALIDATION_STEPS
)
logging.info('TREINAMENTO DO MODELO FINALIZADO')

In [ ]:
# Save it for later
print('Saving Model')
model.save("nude_mobilenet2." + str(width) + "x" + str(height) + ".h5")
logging.info('MODELO EXPORTADO COM SUCESSO')

In [ ]:
final_loss, final_accuracy = model.evaluate(validation_generator, steps = constants.VALIDATION_STEPS)
logging.info('AVALIACAO DO MODELO REALIZADO COM SUCESSO')


In [ ]:
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

logging.info('FINAL LOSS: {:.2f}'.format(final_loss))
logging.info('FINAL ACCURACY: {:.2f}%'.format(final_accuracy * 100))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
saved_model_dir = 'nude_mobilenetv2_train'
Path(saved_model_dir).mkdir(parents=True, exist_ok=True)
tf.saved_model.save(model, saved_model_dir)
keras_model_path = os.path.join(saved_model_dir, "saved_model.h5")
weights_path = os.path.join(saved_model_dir, "saved_model_weights.h5")
model.save(keras_model_path)
model.save_weights(weights_path)
print("SavedModel model exported to", saved_model_dir)
logging.info('MODELO SALVO NA PASTA %s', saved_model_dir)

In [ ]:
Path(saved_model_dir).mkdir(parents=True, exist_ok=True)
tf.io.write_graph(graph_or_graph_def=frozen_graph,
logdir=saved_model_dir,
name="frozen_graph.pb",
as_text=False)

logging.info('FROZEN GRAPH SALVOS NA PASTA %s', saved_model_dir)

In [ ]:
indexed_labels = [(index, label) for label, index in train_generator.class_indices.items()]
print(indexed_labels)
logging.info('LABELS: %s', indexed_labels)

In [ ]:
sorted_indices, sorted_labels = zip(*sorted(indexed_labels))
print(sorted_indices)
print(sorted_labels)

logging.info('SORTED INDICES: %s', sorted_indices)
logging.info('SORTED LABELS: %s', sorted_labels)


In [ ]:
labels_dir_path = os.path.dirname(saved_model_dir)
# Ensure dir structure exists
Path(saved_model_dir).mkdir(parents=True, exist_ok=True)
with tf.io.gfile.GFile('labels.txt', "w") as f:
    f.write("\n".join(sorted_labels + ("",)))
print("Labels written to", saved_model_dir)
logging.info('ARQUIVO DE LABELS SALVO COM SUCESSO')